In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
np.random.seed(0)
import random as rn
rn.seed(0)
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization
tf.random.set_seed(0)

2024-01-07 22:58:12.446137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 22:58:12.573034: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Loading processed and curated dataset:
dfHeart = pd.read_csv("/srv/heart_ConditionalMeanImputation.csv")
dfHeart

,Sex_F,Sex_M,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,ExerciseAngina_N,...,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,40,140.0,289.0,0,172,0.0,0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,49,160.0,180.0,0,156,1.0,1
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,37,130.0,283.0,0,98,0.0,0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,48,138.0,214.0,0,108,1.5,1
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,54,150.0,195.0,0,122,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,45,110.0,264.0,0,132,1.2,1
914,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,68,144.0,193.0,1,141,3.4,1
915,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,57,130.0,131.0,0,115,1.2,1
916,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,57,130.0,236.0,0,174,0.0,1


In [3]:
dfHeart["Name"]="name"
# Estimated (from raw data) slices for each hospital
dfHeart.loc[0:293,"Name"] = "hung" # Hungarian Institute of Cardiology, Budapest
# Author: Andras Janosi, M.D.
dfHeart.loc[293:477,"Name"] = "swit" # University Hospital (Zurich and Basel) Switzerland
# Author: William Steinbrunn, M.D. and Matthias Pfisterer, M.D respectively.
dfHeart.loc[477:615,"Name"] = "long" # V.A. Medical Center, Long Beach
# Author: Robert Detrano, M.D., Ph.D.
dfHeart.loc[615:766,"Name"] = "stat" # Statlog dataset (confidential source)
# Author: "King RD. Department of Statistics and Modelling Science, University of Strathclyde, Glasgow"
dfHeart.loc[766:918,"Name"] = "clev" # Cleveland Clinic Foundation
# Author: Robert Detrano, M.D., Ph.D.

In [4]:
dfHeart.columns

Index(['Sex_F', 'Sex_M', 'ST_Slope_Down', 'ST_Slope_Flat', 'ST_Slope_Up',
       'ChestPainType_ASY', 'ChestPainType_ATA', 'ChestPainType_NAP',
       'ChestPainType_TA', 'ExerciseAngina_N', 'ExerciseAngina_Y',
       'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST', 'Age',
       'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'HeartDisease', 'Name'],
      dtype='object')

In [6]:
# Group by 'Name' and get the count of all entries
total_cases_per_hospital = dfHeart.groupby('Name')['HeartDisease'].count()

# Group by 'Name' and sum the 'HeartDisease' column
positive_cases_per_hospital = dfHeart.groupby('Name')['HeartDisease'].sum()
print(positive_cases_per_hospital)

Name
clev     69
hung    106
long    105
stat     70
swit    158
Name: HeartDisease, dtype: int64


In [8]:
total_cases_per_hospital

Name
clev    152
hung    293
long    138
stat    151
swit    184
Name: HeartDisease, dtype: int64

In [7]:
# Calculate negative cases per hospital
negative_cases_per_hospital = total_cases_per_hospital - positive_cases_per_hospital

print(negative_cases_per_hospital)


Name
clev     83
hung    187
long     33
stat     81
swit     26
Name: HeartDisease, dtype: int64


In [ ]:
# Define the number of rows and columns for the subplots
nrows, ncols = 3,2

# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))

# Flatten the axes array
axes = axes.flatten()
dfHeart_temp = dfHeart[Name == "hung"] 
# Select the columns to plot
df_vp = dfHeart_temp.loc[:,['Age','RestingBP','Cholesterol','MaxHR','Oldpeak','HeartDisease']]


# Loop over the columns of the DataFrame and plot each column in a separate subfigure
for i, col in enumerate(df_vp.columns):
    sns.violinplot(x="HeartDisease", y =col, data=df_vp, ax=axes[i])
    
# Set the title of each subfigure to the name of the corresponding column
for ax, col in zip(axes, df_vp.columns):
    ax.set_title(col, fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the number of rows and columns for the subplots
nrows, ncols = 3,2

# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))

# Flatten the axes array
axes = axes.flatten()
dfHeart_temp = dfHeart[Name == "swit"] 
# Select the columns to plot
df_vp = dfHeart_temp.loc[:,['Age','RestingBP','Cholesterol','MaxHR','Oldpeak','HeartDisease']]


# Loop over the columns of the DataFrame and plot each column in a separate subfigure
for i, col in enumerate(df_vp.columns):
    sns.violinplot(x="HeartDisease", y =col, data=df_vp, ax=axes[i])
    
# Set the title of each subfigure to the name of the corresponding column
for ax, col in zip(axes, df_vp.columns):
    ax.set_title(col, fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the number of rows and columns for the subplots
nrows, ncols = 3,2

# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))

# Flatten the axes array
axes = axes.flatten()
dfHeart_temp = dfHeart[Name == "long"] 
# Select the columns to plot
df_vp = dfHeart_temp.loc[:,['Age','RestingBP','Cholesterol','MaxHR','Oldpeak','HeartDisease']]


# Loop over the columns of the DataFrame and plot each column in a separate subfigure
for i, col in enumerate(df_vp.columns):
    sns.violinplot(x="HeartDisease", y =col, data=df_vp, ax=axes[i])
    
# Set the title of each subfigure to the name of the corresponding column
for ax, col in zip(axes, df_vp.columns):
    ax.set_title(col, fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the number of rows and columns for the subplots
nrows, ncols = 3,2

# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))

# Flatten the axes array
axes = axes.flatten()
dfHeart_temp = dfHeart[Name == "stat"] 
# Select the columns to plot
df_vp = dfHeart_temp.loc[:,['Age','RestingBP','Cholesterol','MaxHR','Oldpeak','HeartDisease']]


# Loop over the columns of the DataFrame and plot each column in a separate subfigure
for i, col in enumerate(df_vp.columns):
    sns.violinplot(x="HeartDisease", y =col, data=df_vp, ax=axes[i])
    
# Set the title of each subfigure to the name of the corresponding column
for ax, col in zip(axes, df_vp.columns):
    ax.set_title(col, fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the number of rows and columns for the subplots
nrows, ncols = 3,2

# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))

# Flatten the axes array
axes = axes.flatten()
dfHeart_temp = dfHeart[Name == "clev"] 
# Select the columns to plot
df_vp = dfHeart_temp.loc[:,['Age','RestingBP','Cholesterol','MaxHR','Oldpeak','HeartDisease']]


# Loop over the columns of the DataFrame and plot each column in a separate subfigure
for i, col in enumerate(df_vp.columns):
    sns.violinplot(x="HeartDisease", y =col, data=df_vp, ax=axes[i])
    
# Set the title of each subfigure to the name of the corresponding column
for ax, col in zip(axes, df_vp.columns):
    ax.set_title(col, fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()